In [1]:
import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import test_func as tf
%matplotlib inline



In [2]:
files=glob.glob('train/*.csv')
data_list=[]

for file in files:
    data_list.append(pd.read_csv(file,index_col=0))
    
df=pd.concat(data_list)    

C:\Users\fujir\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [4]:
def data_pre(df):
    df=df.drop(['市区町村名'],axis=1)

    df=df.drop(['種類'],axis=1)

    nonnull_list=[]

    for col in df.columns:
        nonnull=df[col].count()
        if nonnull==0:
            nonnull_list.append(col)
    df=df.drop(nonnull_list,axis=1)   

    dis={
        '30分?60分':45,
        '1H?1H30':75,
        '1H30?2H':105,
        '2H?':120
    }
    df['最寄駅：距離（分）']=df['最寄駅：距離（分）'].replace(dis).astype(float)

    df['面積（㎡）']=df['面積（㎡）'].replace({'2000㎡以上':2000}).astype(float)

    y_list={}
    for i in df['建築年'].value_counts().keys():
        if '平成' in i:
            num=float(i.split('平成')[1].split('年')[0])
            year=33-num
        if '令和' in i:
            num=float(i.split('令和')[1].split('年')[0])
            year=3-num 
        if '昭和' in i:
            num=float(i.split('昭和')[1].split('年')[0])
            year=96-num     
        y_list[i]=year
        y_list['戦前']=76
    df['建築年']=df['建築年'].replace(y_list)   

    year={
        '年第１四半期':'.25',
        '年第２四半期':'.50',
        '年第３四半期':'.75',
        '年第４四半期':'.99',
    }
    year_list={}
    for i in df['取引時点'].value_counts().keys():
        for k, j in year.items():
            if k in i:
                year_rep=i.replace(k, j)
        year_list[i]=year_rep    
    df['取引時点']=df['取引時点'].replace(year_list).astype(float)
    
    for col in ["都道府県名", "地区名", "最寄駅：名称", "間取り", "建物の構造", "用途", "今後の利用目的", "都市計画", "改装", "取引の事情等"]:
        df[col]=df[col].astype('category')
        
    return df

In [5]:
df=data_pre(df)

In [6]:
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error as mae

In [7]:
df_train,df_val=train_test_split(df,test_size=0.2)

In [13]:
col='取引価格（総額）_log'

train_y=df_train[col]
train_x=df_train.drop(col,axis=1)

val_y=df_val[col]
val_x=df_val.drop(col,axis=1)

trains=lgb.Dataset(train_x,train_y)
valids=lgb.Dataset(val_x,val_y)

params={
    'objective':'regression',
    'metrics':'mae'
}
model=lgb.train(params,trains,valid_sets=valids,num_boost_round=1000,early_stopping_rounds=100)

C:\Users\fujir\anaconda3\lib\site-packages\lightgbm\engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.033049 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 11723
[LightGBM] [Info] Number of data points in the train set: 509880, number of used features: 17
[LightGBM] [Info] Start training from score 7.217208


C:\Users\fujir\anaconda3\lib\site-packages\lightgbm\basic.py:1780: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
C:\Users\fujir\anaconda3\lib\site-packages\lightgbm\basic.py:1513: UserWarning: categorical_column in param dict is overridden.
  _log_warning(f'{cat_alias} in param dict is overridden.')


[1]	valid_0's l1: 0.253448
Training until validation scores don't improve for 100 rounds
[2]	valid_0's l1: 0.236944
[3]	valid_0's l1: 0.222467
[4]	valid_0's l1: 0.209756
[5]	valid_0's l1: 0.198529
[6]	valid_0's l1: 0.188607
[7]	valid_0's l1: 0.17977
[8]	valid_0's l1: 0.171865
[9]	valid_0's l1: 0.164859
[10]	valid_0's l1: 0.158553
[11]	valid_0's l1: 0.152927
[12]	valid_0's l1: 0.147961
[13]	valid_0's l1: 0.143604
[14]	valid_0's l1: 0.139741
[15]	valid_0's l1: 0.136067
[16]	valid_0's l1: 0.132814
[17]	valid_0's l1: 0.129837
[18]	valid_0's l1: 0.127227
[19]	valid_0's l1: 0.124587
[20]	valid_0's l1: 0.122441
[21]	valid_0's l1: 0.12037
[22]	valid_0's l1: 0.118132
[23]	valid_0's l1: 0.116426
[24]	valid_0's l1: 0.114586
[25]	valid_0's l1: 0.113021
[26]	valid_0's l1: 0.111441
[27]	valid_0's l1: 0.11013
[28]	valid_0's l1: 0.108795
[29]	valid_0's l1: 0.107576
[30]	valid_0's l1: 0.106523
[31]	valid_0's l1: 0.105627
[32]	valid_0's l1: 0.104797
[33]	valid_0's l1: 0.103945
[34]	valid_0's l1: 0.10306

In [16]:
vals=model.predict(val_x)
mae(vals,val_y)

0.07642224322433727

In [17]:
df_test=pd.read_csv('test.csv',index_col=0)
df_test=data_pre(df_test)

In [19]:
predict=model.predict(df_test)
df_test[col]=predict

In [22]:
pd.DataFrame(model.feature_importance(),index=val_x.columns,columns=['importance']).sort_values(by=['importance'],ascending=False)

,importance
地区名,8549
最寄駅：名称,6351
建築年,3574
取引時点,3062
面積（㎡）,2133
最寄駅：距離（分）,1969
間取り,863
市区町村コード,728
都道府県名,593
改装,529
